<a href="https://colab.research.google.com/github/mojtabaSefidi/Event-Detection-from-Twitter/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisites

In [1]:
!git clone https://github.com/mojtabaSefidi/Event-Detection-from-Twitter.git

Cloning into 'Event-Detection-from-Twitter'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 68 (delta 18), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (68/68), 227.73 KiB | 1.20 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
%cd /content/Event-Detection-from-Twitter

/content/Event-Detection-from-Twitter


In [3]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 31.3 MB/s eta 0:00:00
  Preparing metad

## Dataset


In this section you should specify the dataset in which you want to detect the events. Your proposed dataset should at least has these two columns. if you don't have your own dataset, you can download ours.

1) text

2) datetime

3) hashtag (opitonal)

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
from utils import read_dataset, get_hdbscan

final_dataset = read_dataset(dataset_path = '/content/gdrive/MyDrive/Event Detection from Twitter/Event Detection Final Dataset/20Feb_20March_Processed.csv', # path to dataset
                             datetime_column = 'datetime', # name of datetime column
                             hashtag_column= 'hashtags' # name of hashtag column
                             )

clustering_model = get_hdbscan() # hdbscan clustering model (you can use any other clustering models)

## Trend Detection

For this section, you need a trained model for trend detection, an embedding matrix for embedding layer, a tokenizer and vocab size. You can also train the trend detection in your labeled dataset or even implement your proposed trend detection model and use it in the trending data extraction component. Note that it's advisable to use our proposed architecture and train the model in your own dataset.

In [ ]:
import shutil

!gdown -q 1-7ueGM5CFh1R3oHhMctDStqrib821K4L
!gdown -q 1-9AyraMu1oZKWsw6HDWQmWNmE3nu-dVK
!gdown -q 1-RUNaJioBCAR204RWBagoD5rqvKJo3nT
!gdown -q 1pKIxul6UADQyx7TaaCAkUkw6OSXQDWiu

!unzip -q /content/EventDetection/Model_Checkpoints.zip
!rm '/content/EventDetection/Model_Checkpoints.zip'

shutil.move("/content/EventDetection/content/gdrive/MyDrive/Model Checkpoints", "/content/EventDetection")
shutil.rmtree('/content/EventDetection/content')

In [ ]:
import pickle
with open('text2sequence_tokenizer.pkl','rb') as f:
  text2sequence_tokenizer = pickle.load(f)
vocab_size=len(text2sequence_tokenizer.word_index)+1

with open('embedding_matrix_trend_detection.pkl','rb') as f:
  embedding_matrix = pickle.load(f)

with open('X_test_trend_detection.pkl','rb') as f:
  X_test_sequence = pickle.load(f)
  max_length = X_test_sequence.shape[1]

model_checkpoint_path = 'Model Checkpoints/TrendDetection.ckpt'

In [ ]:
from Trend_Detection.predict import Trending_Data_Extractor
trending_data_extractor = Trending_Data_Extractor(embedding_matrix, # embedding matrix
                                                  model_checkpoint_path, # trend detection model
                                                  text2sequence_tokenizer, # tokenizer
                                                  max_length # maximum path
                                                  )

## Main pipeline

In [6]:
from Event_Detection.entity_clustering import Entity_Clustering
from Event_Detection.entity_extraction import Extract_Entities
from Event_Detection.event_detection_pipeline import Event_Detection_Pipeline

In [7]:
%%time
main = Event_Detection_Pipeline(
    object_entity_extraction = Extract_Entities(dataset=final_dataset, #dataset
                                               embedding_model_path = 'HooshvareLab/bert-fa-zwnj-base', # word embedding model path
                                               NER_model_path = 'HooshvareLab/bert-base-parsbert-ner-uncased', # NER model path
                                               trending_data_extractor = None # trending data extractor object (not necessary)
                                               ),

    object_entity_clustering=Entity_Clustering(clustering_model), # clustering model
    start_date='2023-02-28', # start date
    end_date='2023-03-14', # end date
    result_storage_path='/content/Results', # path for storing the results
    window_size='1D', # window size, for ex : 1D/ 2D/ 1W/ ...
    window_length=500, # number of tweet in each window
    n_common_entity_threshold=3, # minimum number of entity to link the nodes
    # clusters_dict_saving_path = 'AllClusters.pkl', #path for saving the clusters
    segmented_dfs_directory='/content/gdrive/MyDrive/Event Detection from Twitter/Final Results/Detail_EnrichEvent3/Segmented_Dfs', # path of massage block (if available)
    # segmented_dfs_saving_path = 'Segmented_Dfs', # path directory to store the segmented data-frames
    name_entity_loading_path = '/content/gdrive/MyDrive/Event Detection from Twitter/Final Results/Detail_EnrichEvent3/AllNameEntities.pkl', # path of detected entities (if available)
    # NE_occurrence_dict_saving_path = None, # path for saving the detected entities
    filter_trends=False, # turn on/off trending data extraction component
    trending_threshold=0.1, # minimum thresholf for trending data extraction component
    visualize=False # visualize feature space or not
    )


Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


---------------------------------------- Working on window 1 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. Calculate embedding matrix...
4. Cluster name entities...
==== -1 / 95
---------------------------------------- Elapsed Time : 00:00:10
---------------------------------------- Working on window 2 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. Calculate embedding matrix...
4. Cluster name entities...
==== -1 / 77
---------------------------------------- Elapsed Time : 00:00:06
---------------------------------------- Working on window 3 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. Calculate embedding matrix...
4. Cluster name entities...
==== -1 / 90
---------------------------------------- Elapsed Time : 00:00:08
---------------------------------------- Working on window 4 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. 

## Generating final output

In [9]:
from Event_Summarization.predict import Summary_Generator
from Output_Generation.output_generator import Output_Generator

In [10]:
output_generator = Output_Generator(summarization_model_path='/content/gdrive/MyDrive/Event Detection from Twitter/Models and variables/Summerization2.bin', # checkpoint of summarization moedl
                                    gpt_model_path = 'flax-community/gpt2-medium-persian', # path of language model
                                    embedding_model_path='HooshvareLab/bert-fa-zwnj-base' # path of embedding model
                                    )

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
output_generator.export(results_path='/content/Results') # path of results directory

-------------------------- Working on event 1 --------------------------
1) Drawing WordCloud...
2) Generating Summary...
   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...
3) Extract Events' Attributs...
-------------------------- Working on event 2 --------------------------
1) Drawing WordCloud...
2) Generating Summary...
   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...
3) Extract Events' Attributs...
-------------------------- Working on event 3 --------------------------
1) Drawing WordCloud...
2) Generating Summary...
   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...
3) Extract Events' Attributs...
----------------

## Event Summarization

You can also fine-tune your own summarization model using the scripts in synthetic_dataset.py and train.py

In [12]:
summary_genarator = Summary_Generator(summarization_model_path='/content/gdrive/MyDrive/Event Detection from Twitter/Models and variables/Summerization2.bin', # summarization model checkpoint
                                      gpt_model_path = 'flax-community/gpt2-medium-persian', # language model path
                                      embedding_model_path='HooshvareLab/bert-fa-zwnj-base' # embedding model path
                                      )

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
dict_summary, list_summary = summary_genarator.generate_event_summary(event_path='/content/Results/event_6') # path to event directory
dict_summary

   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...


{0: ([0.6583, 0.6239, 0.5865, 0.6728], 0.63538),
 1: ([0.608, 0.6614, 0.6052, 0.6516], 0.63155),
 3: ([0.642, 0.4882, 0.4123, 0.6926], 0.55878),
 4: ([0.5395, 0.3226, 0.2898, 0.5256], 0.41937),
 2: ([0.4967, 0.2597, 0.2587, 0.4636], 0.36968)}

In [14]:
list_summary

['تجزیه طلب تجزیه طلبی آذربایجان غربی حمله شیمیایی مدارس تهران حمله تروریستی مدرسه دخترانه خیابان وصال شیرازی تبریز ایران اینترنشنال منبع ترکیه ای حزب جمهوری اسلامی میگه سناریوسازی کنن ارتش حکومت نظامی میکنه مردم ریختن جلوی درب دانشگاه بقیه ماجرا اشاره میکنن موضوع چیه دوست نداره بفهمه طرف مقابلم یه عده هستن تروریستن تحلیل کنه فکر نمیکنه اگه این خبر منتشر',
 'روابط ایران با ترکیه روابط خوبی نداریمهزار نفر بسیج توسط ارتش جمهوری اسلامی گروهی تروریستی مسلح علیه مردم منطقه قفقاز فرستاده بد اشکالی نداره دوستیعربستان دولتی رسانه ای خبری داره اشاره میکنه سناریوسازی میکنن اون موقع یه عده میگن خب ایراد میگیره طرف میگه مشکل چیه اصلا مشکلی وجود نداشته باشه همه چی عادی میشه مگه مساله داعش حل شه خوبه',
 'دلم میخواد برم ترکیه ببینم اونجا دارن چیکار میکنن اگه ایران بودم میرفتم یه سری بزنم شاید دلم باز بشه بتونم دل بکنم خیلی سخته واقعا باید خودمو خالی کنم واسه زندگیم حس میکنم مغزم کار نمیکنه دوست دارم خودم برگردم باکو بشینم خونه تنها باشم اصلا حوصله ندارم کلا فکر میکنه زندگی نمیکنم الان میخوام بخوابم د